[![Notebook Tutorial](../__code/__all/notebook_tutorial.png)](https://neutronimaging.pages.ornl.gov/en/tutorial/)

<img src='../__docs/__all/notebook_rules.png' />

In [1]:
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
% matplotlib notebook
import numpy as np
from scipy.ndimage import convolve

from ipywidgets import interact, widgets

from NeuNorm.normalization import Normalization

# Define input data files 

In [2]:
list_files = glob.glob('/Volumes/my_book_thunderbolt_duo/IPTS/IPTS-Das-Saikat/only_data_of_interest/image_*.tif')
for _file in list_files:
    assert os.path.exists(_file)

# Loading the data 

In [3]:
o_norm = Normalization()
o_norm.load(file=list_files, notebook=True)
data = o_norm.data['sample']['data']

HBox(children=(Label(value='Loading sample', layout=Layout(width='20%')), IntProgress(value=0, max=10), Label(value='Time remaining:', layout=Layout(width='10%')), Label(value=' >> calculating << ')))

# Preview of data 

In [4]:
def plot(index):
    fig, ax = plt.subplots(figsize=(5,5))
    cax = ax.imshow(data[index], cmap='viridis')
    fig.colorbar(cax)
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# Crop 

In [5]:
x0 = 0
x1 = 460
y0 = 232
y1 = 285

crop_data = [_data[y0:y1, x0:x1] for _data in data]

def plot(index):
    fig, ax = plt.subplots(figsize=(12,2))
    cax = ax.imshow(crop_data[index], cmap='viridis')
    fig.colorbar(cax)
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# Trying to reverse signal

Will take the max intensity and reverse all the values

```
new_pixel = max_value - old_pixel
```

In [11]:
max_values = [np.max(_data) for _data in crop_data]
crop_data_reversed = [(_max_value - _data) for (_data, _max_value) in zip(crop_data, max_values)]

def plot(index):
    fig, ax = plt.subplots(figsize=(12,2))
    cax = ax.imshow(crop_data_reversed[index], cmap='viridis')
    fig.colorbar(cax)
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# Basic histogram segmentation 

In [16]:
threshold = np.array(max_values) - 1
segmented_data = []
for _index, _data in enumerate(crop_data_reversed):
    _index = _data <= threshold[_index]
    _data[_index] = np.NaN
    segmented_data.append(_data)
    
def plot(index):
    fig, ax = plt.subplots(figsize=(12,2))
    cax = ax.imshow(segmented_data[index], cmap='viridis')
    fig.colorbar(cax)
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# Profile along x-axis 

In [21]:
profiles_x_axis = np.nansum(segmented_data, axis=1)
def plot(index):
    fig, ax = plt.subplots(figsize=(5,5))
    cax = ax.plot(profiles_x_axis[index])
    
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

# profile along user defined line 

* User defines start and end of line (using #6)
* algorithm will then keep the pixel within a given width (number of pixels) of this line
* profile along the line is display



In [22]:
x0, y0 = 450, 20
x1, y1 = 104, 26

In [34]:
def _intermediates_points(p1, p2):
    """"Return a list of nb_points equally spaced points
    between p1 and p2

    p1 = [x0, y0]
    p2 = [x1, y1]
    """

    # nb_points ?
    nb_points = np.int(3 * max([np.abs(p1[0] - p2[0]), np.abs(p2[1] - p1[1])]))

    x_spacing = (p2[0] - p1[0]) / (nb_points + 1)
    y_spacing = (p2[1] - p1[1]) / (nb_points + 1)

    full_array = [[np.int(p1[0] + i * x_spacing), np.int(p1[1] + i * y_spacing)]
                  for i in range(1, nb_points + 1)]

    clean_array = []
    for _points in full_array:
        if _points in clean_array:
            continue
        clean_array.append(_points)

    return np.array(clean_array)

In [42]:
full_line = _intermediates_points([x0,y0], [x1,y1])[::-1]
x_axis_line = full_line[:,0]

In [81]:
all_profiles_along_line = []
width = 1 # 2 pixels on each side
for _segmented_data in segmented_data:
    _data = []
    for _x_y in full_line:
        _x, _y = _x_y
        _data.append(np.nanmean(_segmented_data[_y-width:_y+width+1, _x]))
    all_profiles_along_line.append(_data)

/Users/j35/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice
  import sys


In [82]:
%matplotlib inline

In [158]:
def plot(index):
    fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(20,20))
    cax = ax[0].imshow(segmented_data[index], cmap='viridis')
    cax = ax[1].imshow(segmented_data[index], cmap='viridis')
    for _x_y in full_line:
        _x = _x_y[0]
        _y = _x_y[1]
        ax[1].scatter([_x], [_y])
    
    _xaxis = np.arange(len(all_profiles_along_line[index])) + x1
    ax[2].plot(_xaxis, all_profiles_along_line[index])
    
    
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>

I can try to plot the length of each line versus beginning and end coordinates

In [141]:
def calculate_length_of_each_segment(profile=[]):

    # replace nan values by -2
    nan_substitute = -2
    profile = np.array(profile)
    profile_index = [np.isnan(profile)]
    profile[profile_index] = nan_substitute
    
    x0 = 0
    current_value = -1
    
    dict_segment = {}
    
    line_number = 0
    start_flag = True
    start_point = True
    end_point = False
    _dict = {}
    
    for _index in np.arange(len(profile)):
        _live_value = profile[_index]
#         print("live_value: {} at index: {}".format(_live_value, _index))
        # not nan value
        # we reached an edge of a segment
        if _live_value != current_value:
#             print("live_value is different than current_value (start_point, end_point):({},{})".format(start_point, end_point))

            current_value = _live_value
            
            if start_flag:
#                 print("-> starting a new line")
                _dict={}
                _dict['start'] = _index
                if current_value == nan_substitute:
                    _dict['value'] = np.NaN
                else:
                    _dict['value'] = current_value
                start_flag = not start_flag
            else:
#                 print("-> finishing a line")
                _dict['end'] = _index-1           
                _dict['length'] = _dict['end'] - _dict['start']
                dict_segment[str(line_number)] = _dict
                line_number += 1
                
#                 print("-> and starting a new one")
                _dict={}
                _dict['start'] = _index
                if current_value == nan_substitute:
                
                    _dict['value'] = np.NaN
                else:
                    _dict['value'] = current_value
                
        current_value = _live_value
        
    _dict['end'] = _index
    _dict['length'] = _dict['end'] - _dict['start']
    dict_segment[str(line_number)] = _dict
        
    return dict_segment

full_dict_array = []
for _profile in all_profiles_along_line:
    full_dict_array.append(calculate_length_of_each_segment(profile=_profile))

In [199]:
def isolate_length_vs_x(dict_array):
    nbr_point = len(dict_array)
    x_axis = []
    y_axis = []
    for _key in dict_array.keys():
        _entry = dict_array[_key]
        if not np.isnan(_entry['value']):
#             x_axis.append(x1+ np.mean([_entry['start'], _entry['end']]))
            _start = _entry['start']
            _end = _entry['end']
            for _x in np.arange(_start, _end+1):
                x_axis.append(_x + x1)
                y_axis.append(_entry['length'])
    
    return (x_axis, y_axis)

length_vs_x = {}
for index, dict_array in enumerate(full_dict_array):
    [x_axis, y_axis] =isolate_length_vs_x(dict_array)
    length_vs_x[str(index)] = {'x_axis': x_axis,
                               'y_axis': y_axis}

In [200]:
# display the length of each
def plot(index):

    fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(10,10))
    
    # preview of crop image
    cax = ax[0].imshow(segmented_data[index], cmap='viridis')

    # display of length of lines
    _xaxis = np.arange(len(all_profiles_along_line[index])) + x1
    ax[1].plot(_xaxis, all_profiles_along_line[index])
    ax[1].set_title("lines of continued max value")
    ax[1].set_ylabel("Counts")
    
 # length of lines vs pixel
    x_axis = length_vs_x[str(index)]['x_axis']
    y_axis = length_vs_x[str(index)]['y_axis']
    ax[2].plot(x_axis, y_axis, '*r')
    ax[2].set_title("length of corresponding lines for each pixel")
    ax[2].set_ylabel("Length of correspoonding line")
    ax[2].set_xlabel("Pixel")
    
interact(plot,
        index=widgets.IntSlider(min=0,
                         max=len(data)-1,
                         value=0,
                         continuous_update=False
                           ))

interactive(children=(IntSlider(value=0, continuous_update=False, description='index', max=9), Output()), _dom_classes=('widget-interact',))

<function __main__.plot>